In [ ]:
!pip install pyspark

In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Food').getOrCreate()

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df =  spark.sparkContext.textFile('drive/My Drive/ABD/indian_food.csv')

In [ ]:
df = spark.read.csv('drive/My Drive/ABD/indian_food.csv',header = True,inferSchema= True)
df.show()
df.printSchema()


+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|          name|         ingredients|      diet|prep_time|cook_time|flavor_profile| course|        state|region|
+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|    Balu shahi|Maida flour, yogu...|vegetarian|       45|       25|         sweet|dessert|  West Bengal|  East|
|        Boondi|Gram flour, ghee,...|vegetarian|       80|       30|         sweet|dessert|    Rajasthan|  West|
|Gajar ka halwa|Carrots, milk, su...|vegetarian|       15|       60|         sweet|dessert|       Punjab| North|
|        Ghevar|Flour, ghee, kewr...|vegetarian|       15|       30|         sweet|dessert|    Rajasthan|  West|
|   Gulab jamun|Milk powder, plai...|vegetarian|       15|       40|         sweet|dessert|  West Bengal|  East|
|        Imarti|Sugar syrup, lent...|vegetarian|       10|       50|         sweet|dessert|  Wes

In [ ]:
#Find out how many unique dishes are present
df.groupBy('name').count().show()

+-----------------+-----+
|             name|count|
+-----------------+-----+
|            Kheer|    1|
|          Poriyal|    1|
|Sabudana Khichadi|    1|
|   Gajar ka halwa|    1|
|     Keerai sadam|    1|
|         Dalithoy|    1|
|         Mihidana|    1|
|          Singori|    1|
|          Uttapam|    1|
|          Gavvalu|    1|
|   Kakinada khaja|    1|
|   Chak Hao Kheer|    1|
|          Pachadi|    1|
|         Vindaloo|    1|
|     Gheela Pitha|    1|
|         Idiappam|    1|
|     Keerai kootu|    1|
|             Idli|    1|
|            Saath|    1|
|            Rabri|    1|
+-----------------+-----+
only showing top 20 rows



In [ ]:
#Which state has more dishes ?

df.select('name','state').groupBy('state').count().show()

+---------------+-----+
|          state|count|
+---------------+-----+
|       Nagaland|    1|
|      Karnataka|    6|
|             -1|   24|
|         Odisha|    7|
|         Kerala|    8|
|     Tamil Nadu|   20|
|   Chhattisgarh|    1|
| Andhra Pradesh|   10|
| Madhya Pradesh|    2|
|         Punjab|   32|
|        Manipur|    2|
|Jammu & Kashmir|    2|
|            Goa|    3|
|        Haryana|    1|
|        Gujarat|   35|
|      Rajasthan|    6|
|          Assam|   21|
|   NCT of Delhi|    1|
|    West Bengal|   24|
|    Maharashtra|   30|
+---------------+-----+
only showing top 20 rows



In [ ]:
#3. How many dishes from state Karnataka?

df.select('name').where(col('state')== 'Karnataka').count()

6

In [ ]:
# 4 List number of unique regions
df.groupBy('region').count().show()

+----------+-----+
|    region|count|
+----------+-----+
|        -1|   13|
|      NULL|    1|
|     South|   59|
|   Central|    3|
|      East|   31|
|      West|   74|
|North East|   25|
|     North|   49|
+----------+-----+



In [ ]:
# 5.Count number of dishes from each region.
df.select('name','region').groupBy('region').count().show()

+----------+-----+
|    region|count|
+----------+-----+
|        -1|   13|
|      NULL|    1|
|     South|   59|
|   Central|    3|
|      East|   31|
|      West|   74|
|North East|   25|
|     North|   49|
+----------+-----+



In [ ]:
#6.List unique 'flavor_profile' and 'course
df.groupBy('flavor_profile','course').count().show()

+--------------+-----------+-----+
|flavor_profile|     course|count|
+--------------+-----------+-----+
|        bitter|      snack|    1|
|         spicy|    starter|    2|
|          sour|main course|    1|
|            -1|      snack|    3|
|            -1|main course|   26|
|         sweet|main course|    3|
|        bitter|main course|    3|
|         spicy|      snack|   35|
|         sweet|    dessert|   85|
|         spicy|main course|   96|
+--------------+-----------+-----+



In [ ]:
#7.Which state has more 'main course'?
main = df.filter(df["course"] == 'main course').groupBy("state").count()
max_count = state.agg({"count": "max"}).collect()[0][0]
max_states = state.filter(state["count"] == max_count)
max_states.show()

#main_course.select('course').filter(col('course') == main_course.agg({'course':'max'}).collect()[0][0]).count()

+------+-----+
| state|count|
+------+-----+
|Punjab|   28|
+------+-----+



In [ ]:
#8.Give the %of dishes from each region.

df.groupBy("region").count()\
.withColumn("percentage", (col("count") / df.count()) * 100).show()

+----------+-----+-------------------+
|    region|count|         percentage|
+----------+-----+-------------------+
|        -1|   13|  5.098039215686274|
|      NULL|    1|0.39215686274509803|
|     South|   59| 23.137254901960784|
|   Central|    3| 1.1764705882352942|
|      East|   31| 12.156862745098039|
|      West|   74|  29.01960784313726|
|North East|   25|  9.803921568627452|
|     North|   49| 19.215686274509807|
+----------+-----+-------------------+



In [ ]:
#9.List the states which has more dishes from each region.

d=df.groupBy("region","name","state").count().distinct()\
.agg(count(col("state")).alias("count"))
d.show()

from pyspark.sql.window import Window
d = df.groupBy("region", "name", "state").count().distinct()
window_spec = Window.partitionBy("region").orderBy(desc("count"))
d.withColumn("rank", rank().over(window_spec))\
.filter(col("rank") == 1).select("region", "state", "count")
d.show()

+-----+
|count|
+-----+
|  255|
+-----+

+----------+-----------------+---------------+-----+
|    region|             name|          state|count|
+----------+-----------------+---------------+-----+
|     North|         Dum aloo|Jammu & Kashmir|    1|
|     North|     Rajma chaval|             -1|    1|
|     North|   Mushroom matar|         Punjab|    1|
|     South|          Poriyal|     Tamil Nadu|    1|
|      West|          Surnoli|    Maharashtra|    1|
|      East|          Pakhala|         Odisha|    1|
|     North|            Kofta|  Uttar Pradesh|    1|
|     North|   Gajar ka halwa|         Punjab|    1|
|      East|           Phirni|         Odisha|    1|
|     North|Mushroom do pyaza|         Punjab|    1|
|      West|     Coconut vadi|    Maharashtra|    1|
|      West|           Sukhdi|    Maharashtra|    1|
|     South|          Biryani|      Telangana|    1|
|      West|       Sutar feni|    Maharashtra|    1|
|        -1|        Pani puri|             -1|    1|
|Nort